In [1]:
from process_mstar_AConvNet import *

In [10]:
project_root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))

mstar_dataset_root = os.path.join(project_root, "datasets/MSTAR")

# Input directory (raw CD structure)
RAW_DATA_DIR = os.path.join(mstar_dataset_root, "mstar_raw_data")

print(RAW_DATA_DIR)

/home/mqtho/ATR-Code/datasets/MSTAR/mstar_raw_data


In [11]:
import os
import glob
from collections import defaultdict

base_path = '/home/mqtho/ATR-Code/datasets/MSTAR/MSTAR_IMG_JSON'

# ============ SIMPLE : Récursif ============
print("=== COMPTAGE SIMPLE (RÉCURSIF) ===\n")

png_files = glob.glob(os.path.join(base_path, '**', '*.png'), recursive=True)
json_files = glob.glob(os.path.join(base_path, '**', '*.json'), recursive=True)

print(f"Total PNG  : {len(png_files)}")
print(f"Total JSON : {len(json_files)}")
print(f"Total fichiers : {len(png_files) + len(json_files)}")


# ============ DÉTAILLÉ : Par dossier ============
print("\n=== COMPTAGE PAR DOSSIER ===\n")

stats = defaultdict(lambda: {'png': 0, 'json': 0})

for root, dirs, files in os.walk(base_path):
    png_count = len([f for f in files if f.endswith('.png')])
    json_count = len([f for f in files if f.endswith('.json')])
    
    if png_count > 0 or json_count > 0:
        relative_path = os.path.relpath(root, base_path)
        stats[relative_path] = {'png': png_count, 'json': json_count}
        print(f"{relative_path:<50} PNG: {png_count:<5} JSON: {json_count:<5}")


# ============ TABLEAU BIEN FORMATÉ ============
print("\n=== TABLEAU FORMATÉ ===\n")
print(f"{'Chemin':<50} {'PNG':<8} {'JSON':<8} {'Total':<8}")
print("-" * 75)

total_png = 0
total_json = 0

for path in sorted(stats.keys()):
    png_count = stats[path]['png']
    json_count = stats[path]['json']
    total = png_count + json_count
    print(f"{path:<50} {png_count:<8} {json_count:<8} {total:<8}")
    total_png += png_count
    total_json += json_count

print("-" * 75)
print(f"{'TOTAL':<50} {total_png:<8} {total_json:<8} {total_png + total_json:<8}")


# ============ PAR DATASET ET SPLIT ============
print("\n=== PAR DATASET ET SPLIT ===\n")

dataset_stats = defaultdict(lambda: defaultdict(lambda: {'png': 0, 'json': 0}))

for root, dirs, files in os.walk(base_path):
    relative_path = os.path.relpath(root, base_path)
    parts = relative_path.split(os.sep)
    
    if len(parts) >= 2:
        dataset = parts[0]
        split = parts[1]
        
        png_count = len([f for f in files if f.endswith('.png')])
        json_count = len([f for f in files if f.endswith('.json')])
        
        dataset_stats[dataset][split]['png'] += png_count
        dataset_stats[dataset][split]['json'] += json_count

for dataset in sorted(dataset_stats.keys()):
    print(f"\n{dataset}:")
    print(f"  {'Split':<10} {'PNG':<10} {'JSON':<10} {'Total':<10}")
    print(f"  " + "-" * 45)
    
    for split in sorted(dataset_stats[dataset].keys()):
        png = dataset_stats[dataset][split]['png']
        json = dataset_stats[dataset][split]['json']
        total = png + json
        print(f"  {split:<10} {png:<10} {json:<10} {total:<10}")
    
    total_dataset_png = sum(s['png'] for s in dataset_stats[dataset].values())
    total_dataset_json = sum(s['json'] for s in dataset_stats[dataset].values())
    print(f"  {'TOTAL':<10} {total_dataset_png:<10} {total_dataset_json:<10} {total_dataset_png + total_dataset_json:<10}")


# ============ VÉRIFICATION (PNG = JSON) ============
print("\n=== VÉRIFICATION ===\n")
if total_png == total_json:
    print(f"✅ OK : {total_png} PNG et {total_json} JSON")
else:
    print(f"⚠️  ATTENTION : {total_png} PNG mais {total_json} JSON")
    print(f"   Différence : {abs(total_png - total_json)}")

=== COMPTAGE SIMPLE (RÉCURSIF) ===

Total PNG  : 18220
Total JSON : 18220
Total fichiers : 36440

=== COMPTAGE PAR DOSSIER ===

OUTLIER/train/confuser/2S1                         PNG: 299   JSON: 299  
OUTLIER/train/confuser/ZIL131                      PNG: 299   JSON: 299  
OUTLIER/train/known/BTR70                          PNG: 233   JSON: 233  
OUTLIER/train/known/BMP2                           PNG: 233   JSON: 233  
OUTLIER/train/known/T72                            PNG: 232   JSON: 232  
OUTLIER/test/confuser/2S1                          PNG: 274   JSON: 274  
OUTLIER/test/confuser/ZIL131                       PNG: 274   JSON: 274  
OUTLIER/test/known/BTR70                           PNG: 196   JSON: 196  
OUTLIER/test/known/BMP2                            PNG: 195   JSON: 195  
OUTLIER/test/known/T72                             PNG: 196   JSON: 196  
EOC-1/train/BRDM2                                  PNG: 298   JSON: 298  
EOC-1/train/ZSU234                                 PNG: 29

In [ ]:
import json
import os
from collections import defaultdict

path = os.path.join(mstar_dataset_root, "MSTAR_IMG_JSON/OUTLIER")

# Dictionary to store class_ids by folder
train_classes = defaultdict(set)
test_classes = defaultdict(set)

# Check train folder
train_path = os.path.join(path, 'train')
for class_folder in os.listdir(train_path):
    class_folder_path = os.path.join(train_path, class_folder)
    if os.path.isdir(class_folder_path):
        for file in os.listdir(class_folder_path):
            if file.endswith('.json'):
                json_path = os.path.join(class_folder_path, file)
                with open(json_path, 'r') as f:
                    data = json.load(f)
                    class_id = data.get('class_id')
                    train_classes[class_folder].add(class_id)

# Check test folder
test_path = os.path.join(path, 'test')
for class_folder in os.listdir(test_path):
    class_folder_path = os.path.join(test_path, class_folder)
    if os.path.isdir(class_folder_path):
        for file in os.listdir(class_folder_path):
            if file.endswith('.json'):
                json_path = os.path.join(class_folder_path, file)
                with open(json_path, 'r') as f:
                    data = json.load(f)
                    class_id = data.get('class_id')
                    test_classes[class_folder].add(class_id)

# Print results
print("="*60)
print("TRAIN SET - class_id values by folder:")
print("="*60)
for folder in sorted(train_classes.keys()):
    print(f"{folder:20s}: {train_classes[folder]}")

print("\n" + "="*60)
print("TEST SET - class_id values by folder:")
print("="*60)
for folder in sorted(test_classes.keys()):
    print(f"{folder:20s}: {test_classes[folder]}")

print("\n" + "="*60)
print("SUMMARY:")
print("="*60)
print(f"Train folders: {len(train_classes)}")
print(f"Test folders: {len(test_classes)}")


FileNotFoundError: [Errno 2] No such file or directory: '/home/mqtho/ATR-Code/datasets/MSTAR/MSTAR_IMG_JSON/OUTLIER/confuser'